construcción arbol 

In [1]:
from binance.cm_futures import CMFutures
from binance.client import Client
from funciones import *
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import ccxt

API_KEY = "3a6Q0unsfYV8RXpehFENffnE4hgbWcNAaRRVjrEM99dUTt05l9vPQR1tCeZ8RTOh"
API_SECRET = "aDW3Bwd0POckjdUrBm68kMYdm5CGpBW4FiWzVopLPkGuSOhqlwLLevy2VjOEkwIL"
symbol = "BTCUSDT"
client = Client(api_key= API_KEY, api_secret= API_SECRET)

lista_datos=[]

def pedir_datos(moneda,client,limite):
    
    candles = client.get_klines(symbol= moneda, interval=Client.KLINE_INTERVAL_1MINUTE, limit = limite)
    price_df = pd.DataFrame(candles, columns=['dateTime',
                                            'open',
                                            'high', 'low',
                                            'close', 'volume',
                                            'closeTime',
                                            'quoteAssetVolume',
                                            'numberOfTrades',
                                            'takerBuyBaseVol',
                                            'takerBuyQuoteVol', 'ignore'])
    price_df.close = pd.to_numeric(price_df.close)

    return price_df["close"]


In [2]:
# averiguar el estado de las posiciones 

def posiciones(api_key,api_secret, moneda):
    exchange = ccxt.binance({
        'apiKey': api_key,
        'secret': api_secret,
        'enableRateLimit': True,
        'options': {
            'defaultType': 'future',
        }
    })
    # Obtener las posiciones abiertas
    try:
        positions = exchange.fapiPrivateGetPositionRisk()
        if positions:
            found_position = False
            for position in positions:
                if position['symbol'] == moneda:
                    try:
                        ROE = position['unRealizedProfit'] / position['initialMargin']
                        print(f"ROE:{ROE}")
                        found_position = True
                        return [True,ROE] 
                    except:
                        print('No hay ROE')
                    
            if not found_position:
                print("No hay posición abierta para el símbolo:", moneda)
                return [False]
        else:
            print("No hay posiciones abiertas.")
    except Exception as e:
        print("Error al obtener posiciones:")
        print(e)


In [9]:
def calcular_bollinger_bands(precios, dias=20, k=2, moneda="BTCUSDT", tolerancia=0.3):
    # Convertir la lista de precios a una serie de Pandas
    precios = pd.Series(precios)

    # Calcular la media móvil de los precios
    sma = precios.rolling(window=dias).mean()

    # Calcular la desviación estándar de los precios
    std = precios.rolling(window=dias).std()

    # Calcular las bandas de Bollinger superior e inferior
    upper_band = sma + (k * std)
    lower_band = sma - (k * std)

    # Devolver un DataFrame con las bandas de Bollinger y la media móvil
    Tabladatos = pd.DataFrame({'Precio': precios, 'Media Móvil': sma, 'Banda Superior': upper_band, 'Banda Inferior': lower_band})
    
    dato = len(precios) - 1
    print(f"Dato= {dato}")
    BS = Tabladatos["Banda Superior"][dato] #Banda superior
    print(f"BS= {BS}")
    BI = Tabladatos["Banda Inferior"][dato] #Banda inferior
    print(f"BI= {BI}")
    UP = precios[-1] #Ultimo Precio
    print(f"Ultimo Precio UP= {UP}")
    
    Ancha = ((BS-BI)/BI)*100
    if Ancha >= tolerancia:
        BandaAncha = True
    else:
        BanchaAncha = False

    if UP > BS:
        Encima=True
        Debajo = False
        venta = True
        print(f"Moneda:{moneda} --> Esta por encima de la banda de bollinger")
        return [True,Encima, Debajo, BandaAncha, venta]
    
    elif UP < BI:
        Encima=False
        Debajo=True
        venta = False
        print(f"Moneda:{moneda} --> Esta por debajo de la banda de bollinger")
        return [True,Encima, Debajo, BandaAncha, venta]
    else:
        return [False]

In [7]:
def EntregaLista(moneda="BTCUSDT", lista_datos = []):
    if len(lista_datos) <1000:
        datos=pedir_datos(moneda,client,1000)
        for element in datos:
            lista_datos.append(element)
        return lista_datos
            

    elif len(lista_datos) <4800:
        dato=pedir_datos(moneda,client,1)                             #entrega una lista de datos
        lista_datos.append(dato)
        return lista_datos

    elif len(lista_datos) >=4800:
        dato=pedir_datos(moneda,client,1)
        lista_datos.pop(0)
        lista_datos.append(dato)
        return lista_datos



In [6]:
def tomarAcciones_conPosicion(Roe, DentroBollinger, venta):
    if Roe > 0:
        print('Tomar ganancia')
    if Roe < 0:
        if DentroBollinger: #TODO: poner funcion bollinger con parametros diferentes
            print('Ok')
        else:
            if venta:
                print('vender')
            else:
                print('comprar')
    

In [10]:
def tomarAcciones_sinPosicion(BandaAncha, DentroBollinger, Encima):
    if not DentroBollinger:
        print('Ok')
    else:
        if BandaAncha:
            if Encima:
                print('Venta')
            else:
                print('Compra')
        else:
            print('Ok')

Armado arbol completo :

In [11]:
API_KEY = "3a6Q0unsfYV8RXpehFENffnE4hgbWcNAaRRVjrEM99dUTt05l9vPQR1tCeZ8RTOh"
API_SECRET = "aDW3Bwd0POckjdUrBm68kMYdm5CGpBW4FiWzVopLPkGuSOhqlwLLevy2VjOEkwIL"
moneda = "BTCUSDT"
client = Client(api_key= API_KEY, api_secret= API_SECRET)

lista_datos=[]

def AnalisisMoneda(moneda, tolerancia, client, lista_datos):
    lista_datos = EntregaLista(moneda,lista_datos)
    ExistePosicion = posiciones(API_KEY,API_SECRET, moneda)
    
    #------------------------------------- rama izquierda arbol --------------------------------------
    bollinger = calcular_bollinger_bands(lista_datos, 20, 2, moneda , 0.3) # [True,Encima, Debajo,BandaAncha, Venta]
    
    if ExistePosicion[0]:
        Roe = ExistePosicion[1]
        if bollinger[0]: #verifica si esta por fuera de bollinger
            if bollinger[3]:#verifica si es banda ancha
                tomarAcciones_conPosicion(Roe, True, bollinger[4])
            else:
                print('Ok')
        else:
            print('Ok')
            
    #------------------------------------- rama derecha arbol --------------------------------------        
    else:
        tomarAcciones_sinPosicion(bollinger[3], bollinger[0], bollinger[1])
                
    return lista_datos
        
        

In [ ]:
lista_datos = []
while True:
    lista_datos = AnalisisMoneda(moneda, 0.3, client, lista_datos)
    AnalisisMoneda(moneda, 0.3, client, lista_datos)